In [1]:
sage


<module 'sage' from '/opt/sagemath-9.3/local/lib/python3.7/site-packages/sage/__init__.py'>

In [2]:
x = var("x")
eqn =  (x-1)^2 
eqn.subs(x == 5)

16

In [3]:
var("x y")
t = (x^2 +y)^2 
t.derivative(x)

4*(x^2 + y)*x

In [4]:
t.derivative(x,2)

12*x^2 + 4*y

In [5]:
# A partir de maintenant je vais commencer à écrire des algorithmes relatifs à la géométrie de Rieman

In [7]:
def base_naturel(expressions,curv_coor):
    base_n=[]
    for i in curv_coor:
        vector_base_naturel = []
        for u in expressions:
            vector_base_naturel.append(u.derivative(i))
        base_n.append(vector_base_naturel)
        
    return base_n

def scalar(x,y):
    if(len(x) == len(y)):
        a=var("jkl")
        result=a-a
        
        for i in range(len(x)):
            #print(a)
            k=x[i]*y[i]
            result= result+k
        #print(result)
        return result.trig_reduce()
    else:
        print(len(x),len(y))
              
def tenseur_metrique(base_naturel):
    l=base_naturel
    result =[]
    for i in base_naturel:
        k=[]
        for u in l:
            j = scalar(i,u)
            
            k.append(j.simplify_full())
            
        result.append(k)
            
    return result 

def chirstoffel_one(tenseur_metrique,curv_coor):
    result=[]
    dim = len(curv_coor)
    
    for k in range(dim):
        cont1=[]
        for j in range(dim):
            cont2=[]
            for i in range(dim):
                g = (1/2)*(
                tenseur_metrique[i][j].derivative(curv_coor[k])+tenseur_metrique[j][k].derivative(curv_coor[i])-\
                tenseur_metrique[k][i].derivative(curv_coor[j]))
                
                g.simplify_full()
                
                cont2.append(g)
            cont1.append(cont2)
        result.append(cont1)
                
    return result 
            
        
def christoffel_two(tenseur_metrique,curv_coor):
    result=[]
    dim = len(curv_coor)
    
    for k in range(dim):
        cont1=[]
        for i in range(dim):
            cont2=[]
            for j in range(dim):
                a = var("jkl")
                a=a-a
                for l in range(dim):
                    if(tenseur_metrique[i][l]!=0):
                        
                        a = a + (1/2)*(1/(tenseur_metrique[i][l]))*(
                        tenseur_metrique[j][l].derivative(curv_coor[k])+tenseur_metrique[l][k].derivative(curv_coor[j])-\
                        tenseur_metrique[k][j].derivative(curv_coor[l]))
                        
                a.simplify_full()
                cont2.append(a)
                
            cont1.append(cont2)
            
        result.append(cont1)   
    
    return result


def tenseur_riem_christ(tenseur_metrique,curv_coor):
    """ Donne le tenseur covariant de Riemann Chirstoffel """
    result=[]
    dim = len(curv_coor)
    
    christ2 = christoffel_two(tenseur_metrique,curv_coor)
    christ1 = chirstoffel_one(tenseur_metrique,curv_coor)
    
    for i in range(dim):
        cont1=[]
        for j in range(dim):
            cont2=[]
            for r in range(dim):
                cont3=[]
                for s in range(dim):
                    a = var("a")
                    a = a - a
                    a = christ2[i][j][s].derivative(curv_coor[r])-christ2[i][j][r].derivative(curv_coor[s])
                    for k in range(dim):
                        a = a - christ2[r][k][j]*christ1[i][k][s]+christ2[s][k][j]*christ2[i][k][r]
                        
                    a.simplify_full()    
                    
                    cont3.append(a)
                cont2.append(cont3)
            cont1.append(cont2)
        result.append(cont1)
        
    return result
                
def tenseur_ricci(tenseur_metrique,curv_coor):
    result=[]
    dim = len(curv_coor)
    
    christ2 =christoffel_two(tenseur_metrique,curv_coor)
    
    for i in range(dim):
        cont1=[]
        for s in range(dim):
            b=var("b")
            b=b-b
            for k in range(dim):
                a = var("a")
                a = a -a
                a = christ2[i][k][s].derivative(curv_coor[k])-christ2[i][k][k].derivative(curv_coor[s])
                for l in range(dim):
                    a = a+christ2[i][l][s]*christ2[k][k][l]-christ2[i][l][k]*christ2[s][k][l]
                b=b+a
                
            b.simplify_full()
            
            cont1.append(b)
        result.append(cont1)
        
    return result
    

def courbure_scalaire(tenseur_metrique,curv_coor):
    R = var("R")
    R = R - R
    dim = len(curv_coor)
    ricci = tenseur_ricci(tenseur_metrique,curv_coor)
    
    for i in range(dim):
        for j in range(dim):
            if(tenseur_metrique[i][j] != 0):
                R = R + ((1/tenseur_metrique[i][j])*ricci[i][j]).trig_reduce()
            else:
                R = R + 0
                
                
    return R.trig_reduce()

def geodesic(curv_coor,christ_2,temps,subs=0):
    result = []
    chr2 = christ_2
    
    for i in range(len(curv_coor)):
        for j in range(len(curv_coor)):
            for k in range(len(curv_coor)):
                for l in range(len(curv_coor)):
                
                    chr2[i][j][k] = chr2[i][j][k].substitute(u1=subs[0],u2=subs[1],u3=subs[2])
        
    for l in range(len(curv_coor)):
        curv_coor[l].substitute(u1=subs[0])
        curv_coor[l].substitute(u2=subs[1])
        curv_coor[l].substitute(u3=subs[2])
        print(curv_coor[l])
        a = curv_coor[l].derivative(temps,2)
        print(a)
        
        jkl = var("jkl")
        jkl = jkl - jkl 
        for i in range(len(curv_coor)):
            for k in range(len(curv_coor)):
                jkl = jkl + christ_2[i][l][k] + curv_coor[i].derivative(temps)*curv_coor[k].derivative(temps)
             
        a=a+jkl
        result.append(a.simplify_full())
        
    return result
                
        
        
            
    
                    
                    
                    

        


In [8]:
base_cur=var("u1 u2 u3")
#[u1*u2*cos(u3),u1*u2*sin(u3),(1/2)*(u2^2-u1^2)]
#[u1*cos(u2),u1*sin(u2)]

base_n = base_naturel([u1*sin(u2)*cos(u3),u1*sin(u2)*sin(u3),u1*cos(u2)],base_cur)
tf = tenseur_metrique(base_n)
christ1=chirstoffel_one(tf,base_cur)
christ2=christoffel_two(tf,base_cur)
riem_christ=tenseur_riem_christ(tf,base_cur)
ricci=tenseur_ricci(tf,base_cur)
cbs=courbure_scalaire(tf,base_cur)
#geo = geodesic(base_cur,christ2,var("t"))

print("\n\nbase_n=",base_n)
print("\n\ntenseur_metrique=",tf)
print("\n\nchrist1=",christ1)
print("\n\nchrist2=",christ2)
print("\n\nriem_chirst=",riem_christ)
print("\n\nricci=",ricci)
print("\n\ncbs=",cbs)
#print("\n\ngeodesic",geo)




base_n= [[cos(u3)*sin(u2), sin(u2)*sin(u3), cos(u2)], [u1*cos(u2)*cos(u3), u1*cos(u2)*sin(u3), -u1*sin(u2)], [-u1*sin(u2)*sin(u3), u1*cos(u3)*sin(u2), 0]]


tenseur_metrique= [[1, 0, 0], [0, u1^2, 0], [0, 0, u1^2*sin(u2)^2]]


christ1= [[[0, 0, 0], [0, u1, 0], [0, 0, u1*sin(u2)^2]], [[0, -u1, 0], [u1, 0, 0], [0, 0, u1^2*cos(u2)*sin(u2)]], [[0, 0, -u1*sin(u2)^2], [0, 0, -u1^2*cos(u2)*sin(u2)], [u1*sin(u2)^2, u1^2*cos(u2)*sin(u2), 0]]]


christ2= [[[0, 0, 0], [0, 1/u1, 0], [0, 0, 1/u1]], [[0, -u1, 0], [1/u1, 0, 0], [0, 0, cos(u2)/sin(u2)]], [[0, 0, -u1*sin(u2)^2], [0, 0, -cos(u2)*sin(u2)], [1/u1, cos(u2)/sin(u2), 0]]]


riem_chirst= [[[[0, 0, 0], [0, 1/u1^2 - 1, 0], [0, 0, -sin(u2)^2 + 1/u1^2]], [[0, -1/u1^2 - 1, 0], [2/u1^2, 0, 0], [0, 0, -u1*cos(u2)*sin(u2) + cos(u2)/(u1*sin(u2))]], [[0, 0, -sin(u2)^2 - 1/u1^2], [0, 0, -u1*cos(u2)*sin(u2) - cos(u2)*sin(u2)/u1], [2/u1^2, u1*cos(u2)*sin(u2) + cos(u2)/(u1*sin(u2)), 0]]], [[[0, 1/u1^2 - 1, 0], [0, 0, 0], [0, 0, -u1*cos(u2)*sin(u2) + cos(

In [12]:
base_curv = var("x1 x2 x3")
zero = var("a") -var("a")
tenseur_metrique = [[1, zero, zero], [zero, 2*x1, zero], [zero, zero, 2*x2]]
christ2=christoffel_two(tf,base_cur)

print(christ2[0][1][1])

1/u1


In [53]:
a = var("c w t")
import math
u1=c
u2=w*t
u3=math.pi/4
print(u1,u2,u3)
geo = geodesic(base_cur,christ2,a[2],[c,w*t,math.pi/4])
print(christ2)
eqs = []
for i in geo:
    eq = i==0
    eq = eq.substitute(u1 = c,u2 =  w*t,u3 = math.pi/4)
    eqs.append(eq)
    
print(geo)

c t*w 0.7853981633974483
u1
0
u2
0
u3
0
[[[0, 0, 0], [0, 1/c, 0], [0, 0, 1/c]], [[0, -c, 0], [1/c, 0, 0], [0, 0, cos(t*w)/sin(t*w)]], [[0, 0, -c*sin(t*w)^2], [0, 0, -cos(t*w)*sin(t*w)], [1/c, cos(t*w)/sin(t*w), 0]]]
[c*cos(t*w)^2 - 2*c, -(c*cos(t*w)*sin(t*w) - 2)/c, 2*(c*cos(t*w) + sin(t*w))/(c*sin(t*w))]


In [36]:
eq1.simplify_full()

-c*sin(t*w)^2 - c == 0

In [38]:
e

cos(k)^2 + sin(k)^2

In [37]:
e.trig_reduce()

1

In [1]:
var("x y")
f = 2*x + 5*y**2 

In [7]:
f.derivative(y,2)

10